# Quantitiative epi-genetics of the _A. thalinana_ root microbiome

Author: Eddy J. Mendoza Galindo

Masters Thesis at LMU from the MEME program

Started in November 2023

-----------IMPORTANT!!! 

To run properly all nextflow scripts, add this to the ".bashrc" file in the home directory:
ref=https://collab.dvb.bayern/display/LMUBEC/Storage+and+Cluster
    
"#magic and poorly documented ENV variable that allows to omit the -M flag for (most) slurm commands"

export SLURM_CLUSTERS=biohpc_gen

--------IMPORTANT !!
To use temporary directories (like Rscripts do) you need to set this enviromental variable:

    export TMPDIR=path/
    
And make sure the path is compatible with your comands!

#### Bisulfite DNA sequencing pipeline, public data

In [84]:
# First, I downloaded public paired-end data for 3 epiRILs and a Col0 WT (SRA PRJNA388579), in two replicates
# eR92, eR150, eR193
! ls -lh taller/ra42hux/bs/*.gz 

-rw-r--r-- 1 ra42hux pn73so 513M Nov 17 12:46 taller/ra42hux/bs/SRR5631373_1.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 629M Nov 17 12:47 taller/ra42hux/bs/SRR5631373_2.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 527M Nov 17 12:47 taller/ra42hux/bs/SRR5631374_1.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 648M Nov 17 12:47 taller/ra42hux/bs/SRR5631374_2.fastq.gz
-rw-r--r-- 1 ra42hux pn73so  32M Nov 17 14:02 taller/ra42hux/bs/SRR5631375_1.fastq.gz
-rw-r--r-- 1 ra42hux pn73so  28M Nov 17 14:02 taller/ra42hux/bs/SRR5631375_2.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 932M Nov 17 12:41 taller/ra42hux/bs/SRR5631376_1.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 1.2G Nov 17 12:41 taller/ra42hux/bs/SRR5631376_2.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 478M Nov 17 12:42 taller/ra42hux/bs/SRR5631378_1.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 580M Nov 17 12:42 taller/ra42hux/bs/SRR5631378_2.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 466M Nov 17 12:43 taller/ra42hux/bs/SRR5631379_1.fastq.gz
-rw-r--r-- 1 ra42hux pn73so 569M Nov 17 12:43 taller/r

In [2]:
# metadata
! cat taller/ra42hux/bs/SraAccList.csv

ID,line,rep
SRR5631373,eR92,rep1
SRR5631374,eR92,rep1
SRR5631375,eR92,rep2
SRR5631376,eR92,rep2
SRR5631378,eR150,rep1
SRR5631379,eR150,rep1
SRR5631380,eR150,rep2
SRR5631381,eR150,rep2
SRR5631382,eR193,rep1
SRR5631383,eR193,rep1
SRR5631385,eR193,rep2
SRR5631386,eR193,rep2
SRR5631389,Col0,rep1
SRR5631390,Col0,rep1
SRR5631391,Col0,rep2
SRR5631392,Col0,rep2


In [3]:
# create a samplesheet for methylseq
! cat taller/ra42hux/bs/SraAccList.csv | tail -n +2 | sed -E 's/(\w+),(\w+),(\w+)/\2_\3,bs\/\1_1.fastq.gz,bs\/\1_2.fastq.gz/g' > taller/ra42hux/bs/public_BS.csv

I added the headers manually as sugested in the documentation of methylseq

In [4]:
! cat taller/ra42hux/bs/public_BS.csv

sample,fastq_1,fastq_2
eR92_rep1,bs/SRR5631373_1.fastq.gz,bs/SRR5631373_2.fastq.gz
eR92_rep1,bs/SRR5631374_1.fastq.gz,bs/SRR5631374_2.fastq.gz
eR92_rep2,bs/SRR5631375_1.fastq.gz,bs/SRR5631375_2.fastq.gz
eR92_rep2,bs/SRR5631376_1.fastq.gz,bs/SRR5631376_2.fastq.gz
eR150_rep1,bs/SRR5631378_1.fastq.gz,bs/SRR5631378_2.fastq.gz
eR150_rep1,bs/SRR5631379_1.fastq.gz,bs/SRR5631379_2.fastq.gz
eR150_rep2,bs/SRR5631380_1.fastq.gz,bs/SRR5631380_2.fastq.gz
eR150_rep2,bs/SRR5631381_1.fastq.gz,bs/SRR5631381_2.fastq.gz
eR193_rep1,bs/SRR5631382_1.fastq.gz,bs/SRR5631382_2.fastq.gz
eR193_rep1,bs/SRR5631383_1.fastq.gz,bs/SRR5631383_2.fastq.gz
eR193_rep2,bs/SRR5631385_1.fastq.gz,bs/SRR5631385_2.fastq.gz
eR193_rep2,bs/SRR5631386_1.fastq.gz,bs/SRR5631386_2.fastq.gz
Col0_rep1,bs/SRR5631389_1.fastq.gz,bs/SRR5631389_2.fastq.gz
Col0_rep1,bs/SRR5631390_1.fastq.gz,bs/SRR5631390_2.fastq.gz
Col0_rep2,bs/SRR5631391_1.fastq.gz,bs/SRR5631391_2.fastq.gz
Col0_rep2,bs/SRR5631392_1.fastq.gz,bs/SRR5631392_2.fastq.gz

## 20/11/23

In [ ]:
# First, I created an environment with nextflow and charliecloud versions that were compatible
## The conda channels need to be in this order to use the conda profile:  [conda-forge, bioconda, defaults]
conda create -n env_nf nextflow=21.10.0 charliecloud=0.27

# to check both work:
nextflow run hello
ch-run -V

In [ ]:
# I ran the nextflow pipeline, inside my scratch and the conda environment
# using a Tmux console
module load tmux/3.2a
tmux

nextflow \
	run \
	nf-core/methylseq \
	--input 'bs/*{1,2}.fastq.gz' \
	--genome TAIR10 \
	--aligner bwameth \
	--comprehensive \
	--clip_r1 3 \
	--clip_r2 3 \
	--three_prime_clip_r1 3 \
	--three_prime_clip_r2 3 \
	--min_depth 3 \
	--outdir methylseq/public_BS/ \
	-profile biohpc_gen \
	-r 1.6.1 

In [ ]:
-[nf-core/methylseq] Pipeline completed successfully-
Completed at: 20-Nov-2023 17:50:34
Duration    : 4h 31m 38s
CPU hours   : 73.3 (0.8% failed)
Succeeded   : 129
Ignored     : 4
Failed      : 4

## 21/11/23

In [2]:
! ls taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/*sorted.markDups.bam

taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631373_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631374_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631375_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631376_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631378_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631379_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631380_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631381_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631382_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631383_1.sorted.markDups.bam
taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/SRR5631385_1.sorted.markDups.bam

In [5]:
# create a tab-sparated table for MethylScore (column headers are not required), having ID and location
# thinking of scratch as my working directory
! tail -n +2 taller/ra42hux/bs/SraAccList.csv | sed -E 's/,/\t/g' | awk '{print $2 "\t" "methylseq/public_BS/bwa-mem_markDuplicates/" $1 "_1.sorted.markDups.bam"}'
! tail -n +2 taller/ra42hux/bs/SraAccList.csv | sed -E 's/,/\t/g' | awk '{print $2 "\t" "methylseq/public_BS/bwa-mem_markDuplicates/" $1 "_1.sorted.markDups.bam"}' > taller/ra42hux/methylseq/public_BS/bwa-mem_markDuplicates/samples.tsv

eR92	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631373_1.sorted.markDups.bam
eR92	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631374_1.sorted.markDups.bam
eR92	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631375_1.sorted.markDups.bam
eR92	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631376_1.sorted.markDups.bam
eR150	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631378_1.sorted.markDups.bam
eR150	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631379_1.sorted.markDups.bam
eR150	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631380_1.sorted.markDups.bam
eR150	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631381_1.sorted.markDups.bam
eR193	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631382_1.sorted.markDups.bam
eR193	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631383_1.sorted.markDups.bam
eR193	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631385_1.sorted.markDups.bam
eR193	methylseq/public_BS/bwa-mem_markDuplicates/SRR5631386_1.sorted.markDups.bam
Col0	methylseq/publi

In [ ]:
# MethylScore won't work if the reference is not the same, so I downloaded the genome from iGenomes
wget https://s3.amazonaws.com/igenomes.illumina.com/Arabidopsis_thaliana/Ensembl/TAIR10/Arabidopsis_thaliana_Ensembl_TAIR10.tar.gz
gunzip Arabidopsis_thaliana_Ensembl_TAIR10.tar.gz
tar -xvf Arabidopsis_thaliana_Ensembl_TAIR10.tar

In [1]:
# NOTE: MethylSeq already does the Deduplication part with Picard (before the methylation calls), so we DON'T HAVE to do it again
# Otherwise MethylScore will crash somehow

In [ ]:
# then run methylscore
tmux new -s ms_grp

nextflow run \
	Computomics/MethylScore \
	--SAMPLE_SHEET=methylseq/public_BS/bwa-mem_markDuplicates/samples.tsv \
	--GENOME=ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome.fa \
    --DO_DEDUP false \
    --PROJECT_FOLDER methylscore \
	-profile biohpc_gen

It takes ~45 min per sample. Output:

In [ ]:
[bd/f0ebc8] process > CONSENSUS:SORT_BAM (eR193)                                            [100%] 16 of 16 ✔
[63/550674] process > CONSENSUS:MERGE_BAM (Col0:[SRR5631391_1.sorted.markDups.sorted.bam... [100%] 4 of 4 ✔
[25/8129a4] process > CONSENSUS:READ_STATISTICS (Col0)                                      [100%] 4 of 4 ✔
[4b/4235f2] process > CONSENSUS:SPLIT_BAM (Col0:4)                                          [100%] 28 of 28 ✔
[79/5deb22] process > CONSENSUS:METHYLDACKEL (Col0:4)                                       [100%] 28 of 28 ✔
[db/351ebf] process > CONSENSUS:MATRIX:BUILD (5)                                            [100%] 7 of 7 ✔
[6e/84fb26] process > SAMPLESHEET:GENERATE (genome_matrix.tsv)                              [100%] 1 of 1 ✔
[6d/d7ea0e] process > MRS:CALL_MRS (eR92:genome_matrix.tsv)                                 [100%] 4 of 4 ✔
[cb/e76867] process > MRS:MR_STATISTICS (eR193)                                             [100%] 4 of 4 ✔
[13/9ad783] process > MRS:SPLIT_MRS (all:batchsize:500)                                     [100%] 1 of 1 ✔
[4d/b4305c] process > DMRS:INDEX (genome_matrix.tsv)                                        [100%] 1 of 1 ✔
[3e/83e485] process > DMRS:CALL_DMRS (CHH:all.MRbatch.94)                                   [100%] 276 of 276 ✔
[81/c50fc5] process > DMRS:MERGE_DMRS (CG:all)                                              [100%] 3 of 3 ✔
WARN: Failed to render DAG file: /dss/lxclscratch/0B/ra42hux/methylscore/MethylScore_graph.png
Completed at: 23-Nov-2023 14:07:07
Duration    : 2h 51m 11s
CPU hours   : 36.6
Succeeded   : 377

In [ ]:
# Now, we can try using Col0 as a reference
tmux new -s ms_ref

nextflow run \
	Computomics/MethylScore \
	--SAMPLE_SHEET=methylseq/public_BS/bwa-mem_markDuplicates/samples.tsv \
	--GENOME=ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome.fa \
    --DO_DEDUP false \
    --PAIRWISE Col0 \
    --PROJECT_FOLDER methylscore_ref \
	-profile biohpc_gen

## 23/11/23

#### After comparing both outputs, I decided it is better to use the default group algorithm instead of the reference-based pairwise mode. In the group mode, all DMRs are, in principle, homologous. In this way, we ca use the DMR bed file to intersect the "genome_matrix.tsv" file to get the mean methylated Cs per sample, per DMR!

NOTE: The second value in the genome matrix [X/X/X/X] is the percentage of methylation.
    
Ex: 40/20.0/1/4 --> 20% in that position

From MethylDackel:

    2/ The methylation percentage rounded to an integer
    3/ The number of alignments/pairs reporting methylated bases
    4/ The number of alignments/pairs reporting unmethylated bases

In [ ]:
# I created a new environment with essential software
conda create -n env_g nano r-base bedtools iqtree samtools vcftools htslib bcftools biscuit plink
# I also installed dplyr and data.table inside R
# I tried to install Tidyverse but some dependencies did not worked

## 24/11/23

# WGBS of 169 epiRILs [PRJNA719410](https://www.ncbi.nlm.nih.gov/bioproject/PRJNA719410)

In [1]:
# To download in parallel
! sbatch scripts/download_raw.sh

Submitted batch job 2247863 on cluster biohpc_gen


In [20]:
! squeue -u ra42hux

CLUSTER: biohpc_gen
             JOBID PARTITION     NAME     USER ST       TIME  NODES NODELIST(REASON)
2247863_[447-500%5 biohpc_ge down_epi  ra42hux PD       0:00      1 (Resources)
       2247863_446 biohpc_ge down_epi  ra42hux  R       0:00      1 hlegr1409n02
       2247863_445 biohpc_ge down_epi  ra42hux  R       0:01      1 hlegr1409n03
       2247863_440 biohpc_ge down_epi  ra42hux  R       0:20      1 hlegr1409n04
       2247863_438 biohpc_ge down_epi  ra42hux  R       0:23      1 hlegr1409n03
       2247863_439 biohpc_ge down_epi  ra42hux  R       0:23      1 hlegr1409n09


In [ ]:
# 415 out of 458 were properly downloaded 
# it is possible that the cluster rejected some of the jobs, which need to be done again
# first I will try with the bam files that were provided by Johannes

## 27/11/23

To get the mean mehtylation rate per DMR per context per sample (MethylRille script)

WORKFLOW:
-- Get methylation rate (mR) per site/sample and intersect to the DMR BEDs, do it for each context (Bash + Bedtools)
sed -E 's/\w+\/(\w+\.\w+)\/\w+\/\w+/\1/g'

ND=$(wc -l DMRs.CG.bed | tr ' ' '\n' | head -1)
paste <(cat DMRs.CG.bed | cut -f 1-3) <(awk -v ND="$ND" 'BEGIN { for (i = 1; i <= ND; i++) printf "dmr_%d_CG%s", i, (i < ND ? "\n" : "") }')
cat DMRs.CG.bed | cut -f 1-3 | awk '{ printf $0 "\t" "dmr_%d_CG\n", NR }' 

bedtools intersect -a <(genome_matrix.bed | awk '$4 == "CG"') -b dmr.bed -wb 

-- Calculate mean mR per DMR and produce matrices (binary and non-binary; in R)

R needs the following columns in a tsv file:
  
      chr
      pos
      pos_rep
      context
      sample_i
    ...
      sample_j
      chr_dmr
      start_dmr
      end_dmr
      id_dmr

-- Create phylogenetic tree (IQTree)

-- Merge files so we have input ready for GWAS (Bash)

# Methylome analysis epiRILs

Location of the data:

/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/

## 22/01/24

In [36]:
# create a tab-sparated table for MethylScore (column headers are not required), having ID and location
# replace the name and the location of the data
! cat taller/ra42hux/lines_with_methylome_data.tsv | sed -E 's/^eR(\w+)\t(\w+)/eR\1\t\2\t\/dss\/dsslegfs01\/pn73so\/pn73so-dss-0000\/becker_common\/read_data\/epiRILs\/\1R_All.bam/g' > taller/ra42hux/methylome_metadata.tsv
! head taller/ra42hux/methylome_metadata.tsv

eR102	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/102R_All.bam
eR103	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/103R_All.bam
eR107	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/107R_All.bam
eR118	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/118R_All.bam
eR12	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/12R_All.bam
eR131	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/131R_All.bam
eR13	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/13R_All.bam
eR146	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/146R_All.bam
eR157	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/157R_All.bam
eR160	present	/dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/160R_All.bam


In [38]:
# create one metadata sheet for all lines and one for the lines in my experiment
! cut -f 1,3 taller/ra42hux/methylome_metadata.tsv > taller/ra42hux/er_data/all/all_metadata.tsv
! grep "present" taller/ra42hux/methylome_metadata.tsv | cut -f 1,3 > taller/ra42hux/er_data/exp/exp_metadata.tsv

In [ ]:
# Run methylscore with all lines
# the bam files were generated with Bismark and they don't contail PCR duplicates
# I repeated it on 29/01/24 using the modified fasta reference to include the mitochondria and the chloroplast
tmux new -s eR_all

source activate env_nf

nextflow run \
	Computomics/MethylScore \
	--SAMPLE_SHEET=er_data/all/all_metadata.tsv \
	--GENOME=ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome_CM.fa \
    --DO_DEDUP false \
    --MR_MIN_COV 5 \
    --PROJECT_FOLDER er_data/all \
	-profile biohpc_gen

In [ ]:
# Now just the lines I used in my experiment
# I repeated it on 29/01/24 using the modified fasta reference to include the mitochondria and the chloroplast

tmux new -s eR_exp

source activate env_nf

nextflow run \
	Computomics/MethylScore \
	--SAMPLE_SHEET=er_data/exp/exp_metadata.tsv \
	--GENOME=ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome_CM.fa \
    --DO_DEDUP false \
    --MR_MIN_COV 5 \
    --PROJECT_FOLDER er_data/exp \
	-profile biohpc_gen

I used a minimum of 5 reads to support a methylation call

Reference https://www.nature.com/articles/nmeth.3152

In [ ]:
# to check or open tmux sessions
tmux ls
tmux attach -t eR_all
tmux attach -t eR_exp

In [ ]:
# I also downloaded the VCF and mC calls from the 1001 genomes/epigenomes
wget https://1001genomes.org/data/GMI-MPI/releases/v3.1/1001genomes_snp-short-indel_with_tair10_only_ACGTN.vcf.gz # SNPs
wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE54nnn/GSE54292/suppl/GSE54292_RAW.tar # GMI methylomes
wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE43nnn/GSE43857/suppl/GSE43857_RAW.tar # Salk Methylomes
    
# and the Optimized reference database of TEs from https://urgi.versailles.inra.fr/Data/Transposable-elements/Arabidopsis

Everything worked well!

In [ ]:
# select some lines to plot frequencies, matrix from all lines
shuf -n 10000 genome_matrix.tsv |  sed -E 's/\w+\/(\w+\.\w+)\/\w+\/\w+/\1/g' > random_sample.tsv

## 26/01/24

#### Recombination calculation

In [ ]:
# clean up of the variants
# >80 % of representation
# biallelic SNPs only
# High quality
# from 10 to 50 of coverage
vcftools --vcf 1001genomes_snp-short-indel_with_tair10_only_ACGTN.vcf --remove-indels --max-missing 0.8 --min-alleles 2 --max-alleles 2 --minQ 25 --min-meanDP 10 --max-meanDP 50 --minDP 10 --maxDP 50 --recode --out bi_snps

After filtering, kept 218371 out of a possible 119146348 Sites

In [ ]:
# estimation of recombination rates, population level
mutation rate Arath: 5.9e−9 (10.1126/science.1180677)
    
# first I installed PyRho and SMC++
conda create -n env_rho phython==3.8 # lower than 3.12
conda install -c conda-forge msprime
git clone https://github.com/popgenmethods/ldpop.git ldpop
pip install ldpop/
git clone https://github.com/popgenmethods/pyrho.git pyrho
pip install pyrho/

# test PyRho
python -m pytest pyrho/tests/tests.py

# it worked well
# I removed the environment since I couldn't install SMC++

In [ ]:
# I will calculate pairwise R for the SNPs in each DMR using plink


## 29/01/24

#### Methylation-aware SNP calling 

In [ ]:
# requires nextflow 20.07.1, so I created a new environment
conda create -n env_snp nextflow=20.07.1 charliecloud=0.30

# run in a Tmux background session
tmux new -s eR_snp

source activate env_snp

nextflow run epidiverse/snp \
    --input /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/read_data/epiRILs/ \
    --reference ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome.fa \
    --output er_data/snps \
    --variants \
    --coverage 4 \
    -config biohpc_gen.config

# there are issues with SAMTOOLs so the pipeline won't work
conda remove --name env_snp --all

# I added biscuit to env_g
conda install bioconda::biscuit

In the alingments, the mitochondrial reads are mapped to "M" and the Chloroplastid ones to "C"
Meanwhile, the genome fasta has "Mt" and "Pt" as their names, respectively
So I created another fasta with the names that match the BAM files:
   
    ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome_CM.f

sed 's/>Pt/>C/g' ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome.fa | sed 's/>Mt/>M/g' > ref/Arabidopsis_thaliana/Ensembl/TAIR10/Sequence/WholeGenomeFasta/genome_CM.fa

In [27]:
# create a file to parallel-run the pre-processing for BISCUIT (from taller)
ls er_data/snps/bam/*All.bam | sed -E 's/.bam$//g' > er_data/snps/key.txt

In [30]:
# prepare data for BISCUIT
# requirements: Sorted and Indexed bam files, Indexed reference
! sbatch scripts/pre_biscuit.sh

Submitted batch job 2303957 on cluster biohpc_gen


In [31]:
# now run biscuit
! sbatch scripts/biscuit.sh

Submitted batch job 2304923 on cluster biohpc_gen


In [15]:
# to extract the SNPs
# I tried two ways:

# the built-in version from BISCUIT
# biscuit vcf2bed -t snp biscuit_all.vcf.gz > only_snps.bed

# to analyize substituion rates
# cat only_snps.bed | awk '{print $0 "\t" $4 $5}' | cut -f 1,2,10 | awk '{print $1"_"$2 "\t" $3}' > substitution.bed

# and a custom script that removes variants with methylation information and ambigous calls
! sbatch scripts/biscuit_extract_snps.sh

# mine was way faster!

Submitted batch job 2309689 on cluster biohpc_gen


In [ ]:
# clean SNPs
vcftools --gzvcf biscuit_snps.vcf.gz --remove-indels --max-missing 0.8 --min-alleles 2 --max-alleles 2 --minQ 25 --min-meanDP 10 --max-meanDP 50 --minDP 10 --maxDP 50 --recode --out biallelic_snps

After filtering, kept 733438 out of a possible 57915573 Sites

In [ ]:
# make bed file of biallelic SNPS
grep -v "#" biallelic_snps.recode.vcf | awk '{print $1 "\t"  $2 "\t" $2 }' > biallelic_snps.bed

# Counts of SNPs per chr:
     1      2      3      4      5      M 
182800 121096 146246 115813 166839    644 

In [ ]:
# clean annotation file and extract upstream and downstream seqs
# I had to remove non-UTF8 characters
cat genes_CM.bed | iconv -f utf-8 -t utf-8 -c | sed -E 's/ID=(AT\w+)\;\N.*$/\1/g' > clean_genes_CM.bed

bedtools flank -i clean_genes_CM.bed -g chr_sizes.txt -l 1000 -r 0 -s | sed 's/gene/upstream_region/g' > upstream.bed
bedtools flank -i clean_genes_CM.bed -g chr_sizes.txt -l 0 -r 1000 -s | sed 's/gene/downstream_region/g' > downstream.bed
cat annotation_CM.bed downstream.bed upstream.bed > extended_annotation.bed

# get SNPs that fail into the different genome elements
# I choose to use only TE, gene and flanking regions
cat tes.bed clean_genes_CM.bed upstream.bed downstream.bed > elements.bed

bedtools intersect -a ../er_data/snps/biallelic_snps.bed -b elements.bed -wb | awk '{print $1 "_" $2 "\t" $11 "\t" $13 }' > ../er_data/snps/snp_intersection_genome_elements.tsv

In [ ]:
# PCA
plink --vcf biallelic_snps.recode.vcf --allow-extra-chr --make-bed --out plink/raw

# Extract linked SNPs

plink --bfile plink/raw --allow-extra-chr --set-missing-var-ids @:# --indep-pairwise 5 10 0.33 --out plink/linked

# Prune the original SNP dataset to leave non-linked SNPs

plink --bfile plink/raw --allow-extra-chr --set-missing-var-ids @:# --extract plink/linked.prune.in --make-bed --out plink/pruned
    
# There's strong LD, when applying those filters only very few SNPs are kept (<100) Even after playing with the filters
# Of course, the population is inbred!

# So I did the PCA with all SNPs
plink --bfile plink/raw --allow-extra-chr --pca --out plink/pca

In [ ]:
# Population statistics

#heterozygosity
vcftools --vcf biallelic_snps.recode.vcf --het --out plink/individual

# recombination
plink --bfile plink/raw --r --out plink/correlation_snps

# haplotype blocks
plink --bfile plink/raw --blocks --out plink/haplotypes

# MAF
plink --bfile plink/raw --freq --out plink/maf_per_site

In [ ]:
# I realized that the substitions are very biased and those SNPs are actually monomorphic, as it was previously described in a paper
# I applied a filter and still obsverved a bias
vcftools --gzvcf biscuit_snps.vcf.gz --remove-indels --max-missing 0.8 --min-alleles 2 --max-alleles 2 --maf 0.1 --recode --out plink/test

After filtering, kept 1667 out of a possible 57915573 Sites

vcftools --vcf plink/test.recode.vcf --remove-indels --max-missing 0.8 --min-alleles 2 --max-alleles 2 --maf 0.1 --minDP 4 --recode --out plink/test_filtered

1.4k SNPs

A>C A>G A>T T>A T>C T>G 
117 859  73 105 193 116 

# I'll go now with Revelio and Freebayes

#### TE polymorphisms

In [ ]:
# installation of McClintock
# first I installed mamba in its own env
# note, the mcclintock env is inside the mamba env
https://github.com/bergmanlab/mcclintock.git
cd mcclintock
source activate mamba
mamba env create -f install/envs/mcclintock.yml --name mcclintock
source activate mmclintock
python3 mcclintock.py --install

# IMPORTANT!!!!
# For some reason, inside the main mcclintock folder, there has to be an identical copy of it with the same name
# so, I copied the whole folder to another one, moved into it and then rename it. Don't copy it directly or you'll make a loop


In [5]:
# spli the bam files into fastq1 and fastq2
! sbatch scripts/re-raw.sh

Submitted batch job 2329655 on cluster biohpc_gen


In [5]:
# I downloaded the Araport11 annotation from Tair
# edit chr names
#! sed -E 's/Chr(\w+)/\1/g' Araport11_GFF3_genes_transposons.current.gff > annotation_CM.gff

# to select only TEs
! awk '$3 ==  "transposable_element" ' taller/ra42hux/ref/annotation_CM.gff > taller/ra42hux/ref/tes.gff
! awk '$3 ==  "transposable_element_gene" ' taller/ra42hux/ref/annotation_CM.gff >> taller/ra42hux/ref/tes.gff
! awk '$3 ==  "transposon_fragment" ' taller/ra42hux/ref/annotation_CM.gff >> taller/ra42hux/ref/tes.gff

# at the end I didn't use it because the consensus families were not linked to the Araport Annotation

In [30]:
# create a key for parallel jobs
! ls taller/ra42hux/er_data/fastq/ | sed -E 's/\_\w.fastq.gz//g' | sort | uniq > taller/ra42hux/er_data/tes/key.txt

In [29]:
# run the pipeline
! rm -r taller/ra42hux/er_data/tes/mcclintock/
! mkdir taller/ra42hux/er_data/tes/mcclintock
! sbatch scripts/mcclintock.sh

Submitted batch job 2310352 on cluster biohpc_gen


It was running but not working either complaining, so I discarted the pipeline

#### 1/02/24

#### 02/02/24

In [18]:
# Clean bead file of TEs
# The database and the annotation should have the same families
! cat taller/ra42hux/ref/tes.bed | sed -E 's/ID.+as\=(\w+)$/\1/g' > taller/ra42hux/ref/clean_tes.bed
! head taller/ra42hux/ref/clean_tes.bed

! echo "How many families do we have?"
! cat taller/ra42hux/ref/clean_tes.bed | cut -f 10| sort | uniq -c | wc -l

! echo "How many families are in the database?"
! grep ">" taller/ra42hux/ref/athaTEref_Opt.lib | sed 's/>//g' | wc -l

1	11896	11976	.	.	+	Araport11	transposable_element	.	ATCOPIA24
1	16882	17009	.	.	-	Araport11	transposable_element	.	ATREP4
1	17023	18924	.	.	+	Araport11	transposable_element	.	ATREP3
1	18330	18642	.	.	-	Araport11	transposable_element	.	ATHATN7
1	55675	56576	.	.	+	Araport11	transposable_element	.	SIMPLEHAT1
1	76843	77500	.	.	+	Araport11	transposable_element	.	TA11
1	78287	78785	.	.	-	Araport11	transposable_element	.	HELITRONY1D
1	154330	154418	.	.	-	Araport11	transposable_element	.	ATLINE1_3A
1	193673	194263	.	.	-	Araport11	transposable_element	.	ATREP5
1	194263	194300	.	.	-	Araport11	transposable_element	.	ATREP3
How many families do we have?
39080
How many families are in the database?
326


In [24]:
# Select annotations for those TE families present in the database
# The Colot's library has even less consensus families [only 271] https://github.com/baduelp/public/tree/master/SPLITREADER/thaliana
# We get 31056 annotations (46%)
! awk 'FNR==NR{a[$0];next}{if (($10 in a)){print}}' <(grep ">" taller/ra42hux/ref/athaTEref_Opt.lib | sed 's/>//g') taller/ra42hux/ref/clean_tes.bed > taller/ra42hux/ref/known_tes.bed
# 317 families are present (97%)

# now select those FASTA sequences that are in the annotation, to have fully-matching databases

! perl -e ' ($id,$fasta)=@ARGV; open(ID,$id); while (<ID>) { s/\r?\n//; /^>?(\S+)/; $ids{$1}++; } $num_ids = keys %ids; open(F, $fasta); $s_read = $s_wrote = $print_it = 0; while (<F>) { if (/^>(\S+)/) { $s_read++; if ($ids{$1}) { $s_wrote++; $print_it = 1; delete $ids{$1} } else { $print_it = 0 } }; if ($print_it) { print $_ } }; END { warn "Searched $s_read FASTA records.\nFound $s_wrote IDs out of $num_ids in the ID list.\n" } ' <(cut -f 10 taller/ra42hux/ref/known_tes.bed | sort | uniq ) taller/ra42hux/ref/athaTEref_Opt.lib > taller/ra42hux/ref/known_tes_consensus.fa

Searched 326 FASTA records.
Found 317 IDs out of 317 in the ID list.


In [11]:
# I installed bwa in "env_g" to create the bwa indexes for TEMP2
# then I indexed the genome_CM file

# run the script
! sbatch scripts/temp2.sh

Submitted batch job 2330100 on cluster biohpc_gen


In [8]:
# I noticed eR10 had insertions but somehow the pipeline doesn't work 
# So I tried just this sample to test
! sbatch scripts/temp2_eR10.sh

Submitted batch job 2329753 on cluster biohpc_gen


In [6]:
# PUTATIVE INSERTIONS
! find taller/ra42hux/er_data/tes/results -type f -name "*.insertion.raw.bed" -exec wc -l {} + > taller/ra42hux/er_data/tes/raw_insertions.list
! find taller/ra42hux/er_data/tes/results -type f -name "*.insertion.raw.bed" -exec awk 'BEGIN {FS=OFS="\t"} {print $0, FILENAME}' {} + > taller/ra42hux/er_data/tes/raw_insertions.tsv
# results
! find taller/ra42hux/er_data/tes/results -type f -name "*.insertion.bed" -exec wc -l {} + > taller/ra42hux/er_data/tes/insertions.list
! find taller/ra42hux/er_data/tes/results -type f -name "*.insertion.bed" -exec awk 'BEGIN {FS=OFS="\t"} {print $0, FILENAME}' {} + > taller/ra42hux/er_data/tes/insertions.tsv

### Revelio + FreeBayes SNPs

In [32]:
# I installed Freebayes and Pysam for Revelio, Pysam was installed with pip!
# Paper Revelio: https://link.springer.com/article/10.1186/s12864-022-08691-6

! sbatch scripts/revelio.sh

Submitted batch job 2328826 on cluster biohpc_gen


In [7]:
# create bam list file
# ls er_data/snps_rev_bay/masked/*.bam > er_data/snps_rev_bay/bam.list

# Sort and index every BAM so it can match the genome coordinates for multithreading
! sbatch scripts/pre_freebayes.sh

Submitted batch job 2329693 on cluster biohpc_gen


In [9]:
# now run FreeBayes
! sbatch scripts/freebayes.sh

# the compression and sorting was done with a for loop since it needs to be after the multithreading

Submitted batch job 2330050 on cluster biohpc_gen


In [ ]:
# rename VCF since all FreeBayes output are "unknown"
for file in vcf/*.gz;
do
name=$(echo ${file} | sed 's/vcf\///g' | sed 's/.vcf.gz//g');
picard RenameSampleInVcf \
      INPUT=${file} \
      OUTPUT=vcf/named_${name}.vcf \
      NEW_SAMPLE_NAME=${name} ;
done

In [ ]:
for file in vcf/named*.vcf; do echo "$file"; bgzip -f --threads 16 $file; done

In [ ]:
for file in vcf/named*.gz; do echo $file; bcftools index --threads 16 $file; done

In [ ]:
# the merging chrashes when processing many indels
# remove all indels
for file in vcf/named*.gz;
do
name=$(echo ${file} | sed 's/vcf\/named//g' | sed 's/.vcf.gz//g') ;
echo ${name} ;
vcftools --gzvcf $file --remove-indels --recode --out vcf/only_snps.${name}.vcf ;
done

In [ ]:
# now compress and index again
for file in vcf/only_snps*.vcf; do echo "$file"; bgzip -f --threads 16 $file; done
for file in vcf/only_snps*.gz; do echo $file; bcftools index --threads 16 $file; done

In [ ]:
# merge VCF
bcftools merge --file-list only_snps_named_vcf.list -o raw_snps.vcf --threads 16
# it worked!

In [ ]:
# filter biallelic SNPs by quality, MAF and representation
vcftools --vcf raw_snps.vcf --max-missing 0.8 --min-alleles 2 --max-alleles 2 --maf 0.1 --minDP 4 --recode --out filtered
# After filtering, kept 624 out of a possible 145987 Sites

# Without representative SNPS
vcftools --vcf raw_snps.vcf --min-alleles 2 --max-alleles 2 --maf 0.1 --minDP 4 --recode --out biallelic
# After filtering, kept 128095 out of a possible 145987 Sites

# so most of them are missing a lot! (0.004% vs 87%)

Population genetics statistics

In [ ]:
# check Substitution rates for each set
grep -v "^#" filtered.recode.vcf | cut -f 1-5 | awk '{print $1 "\t" $2 "\t" $2 "\t" $4 ">" $5 }' > filtered.bed
grep -v "^#" biallelic.recode.vcf | cut -f 1-5 | awk '{print $1 "\t" $2 "\t" $2 "\t" $4 ">" $5 }' > biallelic.bed

#heterozygosity
vcftools --vcf filtered.recode.vcf --het --out plink/individual

# Load file
plink --vcf filtered.recode.vcf --allow-extra-chr --make-bed --out plink/raw

# Extract linked SNPs

plink --bfile plink/raw --allow-extra-chr --set-missing-var-ids @:# --indep-pairwise 50 10 0.33 --out plink/linked
# 272 of 624 variants removed.

# Prune the original SNP dataset to leave non-linked SNPs
plink --bfile plink/raw --allow-extra-chr --set-missing-var-ids @:# --extract plink/linked.prune.in --make-bed --out plink/pruned
#  352 variants remaining
    
# run pca all SNPs and no-LD SNPs
plink --bfile plink/raw --allow-extra-chr --pca --out plink/pca_all
plink --bfile plink/pruned --allow-extra-chr --pca --out plink/pca_no-ld

# calculate frequency distribution of rare variants [0.01>MAF<0.2] using complete biallelic SNP dataset
plink --vcf biallelic.recode.vcf --allow-extra-chr --make-bed --out plink/full
plink --bfile plink/full --allow-extra-chr --geno 0.2 --maf 0.01 --max-maf 0.2  --make-bed --out plink/rare

# 36 SNPs left!

# calculate counts for the whole data set
plink --bfile plink/full --allow-extra-chr --set-missing-var-ids @:# --freq counts --out plink/maf_per_site 

In [ ]:
# Make a phylogenetic tree
# Took it from here: https://github.com/edgardomortiz/vcf2phylip
# For heterozygous SNPs: "chooses a nucleotide from a heterozygous genotype at random to avoid IUPAC"
python vcf2phylip.py -i taller/ra42hux/er_data/snps_rev_bay/filtered.recode.vcf \
                     --output-folder taller/ra42hux/er_data/snps_rev_bay/ \
                     --output-prefix filtered --fasta -r

# I also created another one with ambious heterozygous calls 
# From IQTREE: each represented character will have equal likelihood
# Why? Because randomly choosing one allele may bias the tree
python vcf2phylip.py -i taller/ra42hux/er_data/snps_rev_bay/filtered.recode.vcf \
                     --output-folder taller/ra42hux/er_data/snps_rev_bay/ \
                     --output-prefix ambigous --fasta -r

In [ ]:
# Create ML trees
iqtree -nt 4 -s ../filtered.min4.fasta -B 1000
iqtree -nt 4 -s ../ambigous.min4.fasta -B 1000

In [1]:
# copy files and compare trees
! cp taller/ra42hux/er_data/snps_rev_bay/ambigous.min4.fasta.contree /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/genetic.tree
! cp taller/ra42hux/er_data/all/rille/phylo.tsv.fa.contree /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/epi-genetic.tree

## Col-0, root vs leaf

In [ ]:
# 2 leaf samples
# 3 root samples, each sequenced in two lanes
# leaf samples have probably more coverage since the files are twice the size
# I used Bismark since it was also used for the epiRILs
# I trimmed 5bp at both ends as Isaac recommended

tmux new -s wt
source activate env_nf

nextflow run \
	nf-core/methylseq \
	--input 'wt/raw/*{1,2}.fastq.gz' \
	--genome TAIR10 \
	--aligner bismark \
	--comprehensive \
	--clip_r1 5 \
	--clip_r2 5 \
	--three_prime_clip_r1 5 \
	--three_prime_clip_r2 5 \
	--min_depth 4 \
	--outdir wt/methylseq/ \
	-profile biohpc_gen \
	-r 1.6.1 

______________________________________________________
Completed at: 05-Feb-2024 17:52:36
Duration    : 1h 30m 50s
CPU hours   : 65.4
Succeeded   : 44

In [27]:
# SNP calling with the data
! sbatch scripts/snps_wt.sh

Submitted batch job 2333715 on cluster biohpc_gen


In [ ]:
# merge anc clean SNPs
bcftools merge --file-list file.list -o raw_snps.vcf --threads 16
vcftools --vcf raw_snps.vcf --max-missing 0.8 --min-alleles 2 --max-alleles 2 --maf 0.1 --minDP 4 --recode --out wt_filtered
# After filtering, kept 3698 out of a possible 90807 Sites
# more SNPs than the epiRIL population!

# check Substitution rates for each set
grep -v "^#" vcf/wt_filtered.recode.vcf | cut -f 1-5 | awk '{print $1 "\t" $2 "\t" $2 "\t" $4 ">" $5 }' > wt_filtered.bed

# calculate counts for the whole data set
plink --vcf vcf/wt_filtered.recode.vcf --double-id --allow-extra-chr --make-bed --out plink/wt_raw
plink --bfile plink/wt_raw --allow-extra-chr --set-missing-var-ids @:# --freq counts --out plink/wt_maf_per_site 

In [4]:
# Check it in the R Studio Server
! cp taller/ra42hux/wt/snps/wt_filtered.bed /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/wt_snps.bed
! cp taller/ra42hux/ref/clean_genes_CM.bed /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/genes.bed

In [26]:
# create metadata for methylscore
! paste <(ls taller/ra42hux/wt/methylseq/bismark_deduplicated/*.bam | sed 's/taller\/ra42hux\///g' | sed -E 's/.+ed\/(\w+\-\w)_.+/\1/g') <(ls taller/ra42hux/wt/methylseq/bismark_deduplicated/*.bam | sed 's/taller\/ra42hux\///g' ) > taller/ra42hux/wt/metadata_for_methylscore.tsv 
! cat taller/ra42hux/wt/metadata_for_methylscore.tsv 

leaf-1	wt/methylseq/bismark_deduplicated/leaf-1_1_val_1_bismark_bt2_pe.deduplicated.bam
leaf-2	wt/methylseq/bismark_deduplicated/leaf-2_1_val_1_bismark_bt2_pe.deduplicated.bam
root-1	wt/methylseq/bismark_deduplicated/root-1_1_val_1_bismark_bt2_pe.deduplicated.bam
root-2	wt/methylseq/bismark_deduplicated/root-2_1_val_1_bismark_bt2_pe.deduplicated.bam
root-3	wt/methylseq/bismark_deduplicated/root-3_1_val_1_bismark_bt2_pe.deduplicated.bam


In [ ]:
# Run methylscore to call methylation
tmux new -s wt
source activate env_nf

nextflow run \
	Computomics/MethylScore \
	--SAMPLE_SHEET=wt/metadata_for_methylscore.tsv \
	--GENOME=ref/genome_CM.fa \
    --DO_DEDUP false \
    --MR_MIN_COV 5 \
    --PROJECT_FOLDER wt/methylscore \
	-profile biohpc_gen

________________________________________-
Completed at: 05-Feb-2024 20:21:00
Duration    : 2h 4m 54s
CPU hours   : 25.3
Succeeded   : 334

In [4]:
# I made scripts to process the output of MethyRille {KingSize}
# copy the R script to bin/ folder of the environment
! cp surco.R .conda/envs/env_g/bin/surco.R
! chmod 701 .conda/envs/env_g/bin/surco.R
! ls -lh .conda/envs/env_g/bin/surco.R

-rwx-----x 1 ra42hux pn73so 2.6K Feb  7 12:27 .conda/envs/env_g/bin/surco.R


In [24]:
# Also copy the file to create a fasta file
! cp tab_to_phy.pl .conda/envs/env_g/bin/tab_to_phy.pl
! chmod 701 .conda/envs/env_g/bin/tab_to_phy.pl
! ls -lh .conda/envs/env_g/bin/tab_to_phy.pl

-rwx-----x 1 ra42hux pn73so 709 Feb  6 16:38 .conda/envs/env_g/bin/tab_to_phy.pl


# MethylRille

In [5]:
# I have to edit each run manually
! sbatch scripts/methyl-rille_wt.sh
! sbatch scripts/methyl-rille_all.sh
! sbatch scripts/methyl-rille_exp.sh

Submitted batch job 2335452 on cluster biohpc_gen
Submitted batch job 2335453 on cluster biohpc_gen
Submitted batch job 2335454 on cluster biohpc_gen


In [1]:
# Copy files to make PCA in R
! cp taller/ra42hux/er_data/all/rille/mean_matrix.tsv /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/all_matrix.tsv
! cp taller/ra42hux/er_data/exp/rille/mean_matrix.tsv /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/exp_matrix.tsv
! cp taller/ra42hux/er_data/all/rille/binary_matrix.tsv /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/all_bin.tsv

#### Cleaning of the DMRs

Create list of "unwanted DMRs"

##### 1.- Clean DMRs that change between roots and leaves

In [7]:
# To do the differential methylation analysis between roots and leaves
! cp taller/ra42hux/wt/rille/mean_matrix.tsv /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/root_leaf_matrix.tsv

In [18]:
# get the list
! cat /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/root_leaf_differential_mC.tsv | awk '$6 == "dm" ' | cut -f 1 > taller/ra42hux/er_data/root_vs_leaf.list

##### 2.- Clean CG DMRs that match with genes with at least 80% of the region

- From 27242 CG DMRs;

- 14770, 15062 and 16395 are covered at the 100, 80 and 10%, respectively, by at least one full gene annotations (all that is transcribed)

- 12570, 13773 and 19774 are covered at the 100, 80 and 10% by an exonic annotation, respectively.

NOT DOING IT

##### 3.- Clean DMRs that overlap with SNPs

NOT DOING IT

-Why? Considering the origin of the epiRIL population, most SNPs should be in  low frequencies and those that are not should then be from the parents

##### 4.- Clean DMRs that never are unmethylated (<15% methylation)

ONLY FOR THE BINARY ANALYSIS

# Amplicon analysis

In [ ]:
# Isntall BaseSpace from anaconda
bs auth
bs download project -i 409471065 -o amplicon/

------- Test
bwa mem -t 16 ../../ref/phix.fa ../Undetermined/Undetermined_S0_L001_R1_001.fastq ../Undetermined/Undetermined_S0_L001_R2_001.fastq | samtools view -@ 16 -b > test.bam
8.7 % Maps to PhiX (~2M reads)

32667186 + 0 in total (QC-passed reads + QC-failed reads)
2869681 + 0 mapped (8.78% : N/A)
32562914 + 0 paired in sequencing
16281457 on each read

-------- Now check overlaps with my indexes
One read is enough since they all should be the same
grep "1:N:0" Undetermined_S0_L001_R1_001.fastq | sed -E 's/.+1\:N\:0\://g' | sort | uniq > indexes.list

Only 98 combinations have either the i5 or the i7???

The top 95 combinations (based on the number of reads) have from 300k to 60k reads! Then it jumps to 11k
It makes sense in terms of samples

--------------
My Forward barcodes went to the reverse read (NNNNNNNN+HHHHHHHH)
And the reverse barcodes in the "reverse complement" form in the forward read (HHHHHHHH+NNNNNNNN)

      8 ACGCTACT
      8 ACTACGAC
      8 ACTCACTG
      8 CGAGAGTT
      9 CGAGCGAC
      8 CTGCGTAG
      8 GACATAGT
      9 GTCTATGA
      9 GTCTGCTA
      8 TAGTCTCC
      9 TATAGCGA
      8 TGAGTACG
        

     12 ACTATCTG
     12 ATCGTACG
     12 CGTGAGTG
     12 CTGCGTGT
     12 GACACCGT
     12 GGATATCT
     12 TAGCGAGT
     12 TCATCGAG

In [ ]:
Demultiplexing using STACKS and the corrected indexes
# ITS
awk '$5 == "its"' ../barcodes_full.tsv | awk '{print $11 "\t" $12 "\t" $8}' > undt_its/its.barcodes
process_radtags -P -p undt_its/ -b undt_its/its.barcodes --index-index -r --disable-rad-check -o undt_its/demux/

# delete empty files
find ./ -size 0 -print -delete

# The 96 lines that have more than 100k in sizes are the ones sequenced
find . -type f -size +100k | wc -l

# 16S pool 1
awk '$5 == "16s"' ../barcodes_full.tsv | awk '$6 == "P1" '| awk '{print $11 "\t" $12 "\t" $8}' > 16s/pool_1.barcodes
process_radtags -P -p pool_1_raw/ -b pool_1.barcodes --index-index -r --disable-rad-check -o demux_pool_1/

# Pool 2
# For the Run:
awk '$5 == "16s"' ../barcodes_full.tsv | awk '$6 == "P2" '| awk '{print $8 "\t" $11 "\t" $12}' > 16s/pool_2_indexes.tsv
grep "BS" 16s/pool_1.barcodes | awk '{print $3 "\t" $1 "\t" $2}' >> 16s/pool_2_indexes.tsv

# for the demultiplex
awk '$5 == "16s"' ../barcodes_full.tsv | awk '$6 == "P2" '| awk '{print $11 "\t" $12 "\t" $8}' > 16s/pool_2.barcodes
grep "BS" 16s/pool_1.barcodes >> 16s/pool_2.barcodes

# Demux
process_radtags -P -p 16s/pool2_raw/ -b 16s/pool_2.barcodes --index-index -r --disable-rad-check --barcode-dist-2 0 -o 16s/pool2_raw/demux/

# delete bad/empty files
find 16s/pool2_raw/demux/ -type f -name "*.gz" -size -4000k -delete

In [3]:
# Copy ITS reads to analyze on R
# remove small files: find . -type f -name "*.fq" -size -100k -delete
! cp taller/ra42hux/amplicon/its/demux/*.fq /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/its/

### Rhizosphere

In [3]:
# copy rhizosphere from pool1 

#find . -type f -name "*.fq" -size -100k -delete
#rm *rem*

! cp taller/ra42hux/amplicon/16s/demux_pool_1/*.gz /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/16s/pool1/

In [2]:
# copy pool2 samples
! cp taller/ra42hux/amplicon/16s/pool2_raw/demux/*.gz /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/16s/pool2/

In [3]:
# make ML trees of the sequences
# ! cp /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/pool1.fa taller/ra42hux/amplicon/trees/pool1.fa

! cp /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/rhizo.fa taller/ra42hux/amplicon/trees/rhizo.fa

In [ ]:
# align and make trees using the model as in https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-016-0992-y
mafft --auto pool1.fa > pool1.aln #FFT-NS-2 
iqtree -nt 16 -s pool1.aln -m GTR+I+G

In [4]:
# I made an script as it would take a time
#mafft --auto rhizo.fa > rhizo.aln
#iqtree -nt 16 -s rhizo.aln -m GTR+I+G -B 1000

! sbatch scripts/tree_rhizo.sh

Submitted batch job 2383474 on cluster biohpc_gen


In [4]:
! sbatch scripts/tree_pool1.sh

Submitted batch job 2370002 on cluster biohpc_gen


In [5]:
! cp taller/ra42hux/amplicon/trees/rhizo.aln.treefile /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/rhizo_tree.nw

#### Endosphere

In [1]:
# Copy sequences to make a tree
! cp /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/endo.fa taller/ra42hux/amplicon/trees/endo.fa

In [ ]:
mafft --auto endo.fa > endo.aln
iqtree -nt 16 -s endo.aln -m GTR+I+G -B 1000

In [2]:
# move the tree to the RStudio server
! cp taller/ra42hux/amplicon/trees/endo.aln.treefile /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/endo_tree.nw

In [1]:
! cp taller/ra42hux/amplicon/trees/endo.aln.contree /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/endo_contree.nw
! cp taller/ra42hux/amplicon/trees/rhizo.aln.contree /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/rhizo_contree.nw

### Looking for the causal gene

In [ ]:
# Download root data
wget https://ftp.ncbi.nlm.nih.gov/geo/series/GSE79nnn/GSE79710/suppl/GSE79710_RAW.tar
tar -xvf GSE79710_RAW.tar
gunzip *.gz

# I removed the controls and the lower columnella

In [1]:
# copy the bed file of the candidates
! cp /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/raw/candidate_dmrs.bed taller/ra42hux/amplicon/candidate/candidate_dmrs.bed

In [ ]:
# As I am mostly interested in genes, I will only look for DMRs that intersect with genes OR the downstream/upstream regions (+-1kb)
bedtools slop -i ../../ref/clean_genes_CM.bed -g ./../ref/chr_sizes.txt -b 1000 > extended_genes.bed

# use 100% of the DMR regions to find overlaps
bedtools intersect -a extended_genes.bed -b candidate_dmrs.bed -F 1 -wb | cut -f 10,15 > intersects.list
# 26 unique genes but 30 total intersections, only 23 DMRs

# select genes from the data and merge them into a file
# include ACTIN2
rm candidates_fpkm.tsv

for i in raw/*fpkm*
do
name=$(echo ${i} | sed -E 's/.+\km_(\w+)\.tsv/\1/g' )
awk 'FNR==NR{a[$0];next}{if (($1 in a)){print}}' <(cat <(echo AT3G18780) <(cut -f 1 intersects.list)) ${i} | awk -v name="$name" '{ printf $0 "\t" name "\n" }' >> candidates_fpkm.tsv
done


# get the methylated sites from all tissues and the 23 intersecting DMRs
# more than 5 reads to process a site
# add actin
awk 'FNR==NR{a[$0];next}{if (($5 in a)){print}}'  <(cut -f 2 intersects.list | sort | uniq) candidate_dmrs.bed | cut -f 1-5 > intersecting_dmrs.bed
grep "AT3G18780" extended_genes.bed  | cut -f 1-4,10 >> intersecting_dmrs.bed

rm candidates_mC.tsv

for i in raw/*mc*
do
name=$(echo ${i} | sed -E 's/.+\ls_(\w+)\.tsv/\1/g' )
bedtools intersect -a intersecting_dmrs.bed -b <(awk '{print $1 "\t" $2 "\t" $2 "\t" $3 "\t" $4 "\t" $5 "\t" $6}' <(tail -n +2 $i | awk '$6 > 5')) -wb | cut -f 5-12 | awk -v name="$name" '{ printf $0 "\t" name "\n" }' >> candidates_mC.tsv
echo "${name} done!"
done

In [4]:
# copy the results to the Server
! cp taller/ra42hux/amplicon/candidate/candidates_mC.tsv /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/candidates_mC.tsv
! cp taller/ra42hux/amplicon/candidate/candidates_fpkm.tsv /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/candidates_fpkm.tsv
! cp taller/ra42hux/amplicon/candidate/intersects.list /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/intersects.tsv

In [ ]:
# get the annotations of the LRR and other elements
grep "dmr_604_CG" intersecting_dmrs.bed > LRR_neighborhood.bed
grep "AT1G10850" ../../ref/clean_genes_CM.bed | cut -f 1-4,10 >> LRR_neighborhood.bed
# 3 SNPs
bedtools intersect -a LRR_neighborhood.bed -b ../../er_data/snps_rev_bay/biallelic.bed -wb | cut -f 6-8 >> LRR_neighborhood.bed
# no TIPs
# no reference TEs
bedtools intersect -a <(bedtools slop -i LRR_neighborhood.bed -g ../../ref/chr_sizes.txt -b 1000) -b ../../ref/clean_tes.bed -F 0.1 | cut -f 1-4,10

In [6]:
! cp taller/ra42hux/amplicon/candidate/LRR_neighborhood.bed /dss/dsslegfs01/pn73so/pn73so-dss-0000/becker_common/eddy/epixbiome/data/LRR_neighborhood.bed